In [120]:
# use Brown corpus movie reviews
from nltk.corpus import movie_reviews as mr
import string
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
import operator
import random
ps = LancasterStemmer()
stop = stopwords.words('english')
# document [0] are the words list, [1] is 'pos' or 'neg' review
documents = [([ps.stem(w.lower()) for w in mr.words(i) if w.lower() not in stop and w.lower() not in string.punctuation], i.split('/')[0]) for i in mr.fileids()]
print('documents:', len(documents))
MAX_WORDS = 200

documents: 2000


In [52]:
# organize unique words and their counts
from collections import Counter
words = []
for d in documents:
    for w in d[0]:
        words.append(w)
word_counts = Counter(words)
print('words:', len(word_counts))
uncommon_words = []
for w in word_counts:
    # focus on least common words
    if 100 > word_counts[w] > 0 and len(w)>2:
        uncommon_words.append(w)
print('# uncommon words:', len(uncommon_words))

words: 21467
# uncommon words: 19786


In [135]:
# map a document to a vector of word indeces
def map_document(document):
    map = [0] * MAX_WORDS
    for i,w in enumerate(document[:MAX_WORDS]):
        if w in uncommon_words:
            map[i] = words.index(w)
    return map

In [152]:
print(map_document(documents[0][0]), documents[0][1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5435, 0, 0, 0, 0, 25023, 0, 0, 0, 0, 0, 165541, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10476, 0, 9490, 0, 0, 0, 0, 0, 0, 36604, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 49454, 0, 0, 0, 0, 137, 0, 0, 0, 109263, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 189085, 0, 140500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 127903, 0, 49516, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39544, 39773, 0, 0, 2083, 0, 0, 0, 0, 0, 0, 0, 566056, 0, 0, 8819, 0, 189085, 0, 0, 0, 0, 0, 11723, 4872, 0, 0, 0, 0, 0, 0, 0, 189085, 566077, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22458, 0, 0, 19254, 189085, 0, 0, 50854, 0, 0, 0, 0, 207038, 0, 19254, 0, 0, 0, 0, 189085, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] pos


In [149]:
# build our data
random.shuffle(documents)
documents = documents[:200]
training_subset = 0.8
data = []
labels = []
for doc in documents[:int(len(documents)*training_subset)]:
    data.append(map_document(doc[0]))
    if doc[1] == 'pos':
        labels.append([0, 1])
    elif doc[1] == 'neg':
        labels.append([1, 0])

In [140]:
import numpy as np
import tflearn

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [150]:
import tensorflow as tf
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, 200])
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

# Define model
model = tflearn.DNN(net)
# Start training (apply gradient descent algorithm)
model.fit(data, labels, n_epoch=1000, batch_size=16, show_metric=True)

Training Step: 2999  | total loss: 10.97868 | time: 0.007s
| Adam | epoch: 1000 | loss: 10.97868 - acc: 0.5232 -- iter: 32/40
Training Step: 3000  | total loss: 11.03211 | time: 0.010s
| Adam | epoch: 1000 | loss: 11.03211 - acc: 0.5209 -- iter: 40/40
--
